# Module Assignment 3 Project for Course BAN6800 (Business Analytics Capstone)

- **Task 0** : Business Analytics Project-Ready Dataset

### <u>1.0 Intro</u>
This assignment is about getting ready the dataset that will be used in the Customer Segmentation project for Cognitive Software Solutions. The company has three branches which produces several products. All the products of the company are grouped into five categories which are (Saas, Cloud Services, Security, Artificial Intelligence and Development Tools). Each of these categories stores data on a seperate server. A total of these different reports is expected to be 100,000 records.

In [223]:
# Import the neccessary libraries

import pandas as pd, warnings
from glob import glob
from sklearn.preprocessing import StandardScaler, LabelEncoder
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

### <u>1.1 Input Data (Data Integration)</u>
First, we need to read all the different dataset from all the five categories and carry out a Data Integration process to consolidate to one dataset.

In [224]:
# Read the directory and fetch all the file names of the category reports using the glob() function to form a list
category_files = glob(pathname="subsidiary_reports/Cognitive*")
print(category_files)

['subsidiary_reports/Cognitive_Development_Tools_Report.csv', 'subsidiary_reports/Cognitive_Saas_Report.csv', 'subsidiary_reports/Cognitive_Security_Report.csv', 'subsidiary_reports/Cognitive_Artificial_Intelligence_Report.csv', 'subsidiary_reports/Cognitive_Cloud_Services_Report.csv']


In [225]:
# Import all the category files and merged them into one file using the concat() function 
dataset = pd.concat((pd.read_csv(filepath_or_buffer=file) for file in category_files), ignore_index=True)

In [226]:
# Confirm if Dataframe has been fully loaded
dataset.head(n=5)

,Customer_ID,Order_ID,Subscription_Date,Product_ID,Product_Name,Category,Subscription_Type,Usage_Frequency,Revenue_Per_Customer,Payment_Plan,Discount_Applied,Support_Tickets,Feature_Adoption,Last_Login,Churn_Status,Customer_Tenure,Customer_Satisfaction,Marketing_Channel,Contract_Renewal,Num_Logins_Last_Month
0,2454.0,1.0,03/07/2023,19,ScriptEase,Development Tools,Basic,Monthly,2097.0,Annual,24,19,36,10/12/2023,0,9,3,Referral,0,25
1,2167.0,5.0,20/09/2022,18,DevSync,Development Tools,Enterprise,Weekly,324.0,Annual,18,33,84,08/05/2023,0,5,5,Referral,0,25
2,2063.0,6.0,26/10/2023,19,ScriptEase,Development Tools,Enterprise,Daily,1701.0,Lifetime,29,28,37,26/10/2023,0,8,2,Referral,0,7
3,4211.0,22.0,22/02/2022,19,ScriptEase,Development Tools,Enterprise,Daily,3649.0,Lifetime,17,27,90,23/06/2023,0,8,2,Direct,1,17
4,1636.0,24.0,03/11/2022,19,ScriptEase,Development Tools,Enterprise,Monthly,2047.0,Annual,25,5,23,12/08/2023,0,9,5,Advertisement,0,14


In [227]:
# Confirm the number of rows and colums to validate the number of expected records
dataset.shape

(100000, 20)

With the shape of the dataset being (100000, 20) has confirmed that all the datasets were integrated and has loaded successfully !

### <u>2.0 Understanding the dataset</u>
Next, we start by preparing the dataset for analysis by doing data cleansing and checking out for inconsistencies.


In [228]:
# Checking out the properties of the dataset
print(dataset.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Customer_ID            99993 non-null   float64
 1   Order_ID               99999 non-null   float64
 2   Subscription_Date      100000 non-null  object 
 3   Product_ID             100000 non-null  int64  
 4   Product_Name           100000 non-null  object 
 5   Category               100000 non-null  object 
 6   Subscription_Type      99998 non-null   object 
 7   Usage_Frequency        100000 non-null  object 
 8   Revenue_Per_Customer   99999 non-null   float64
 9   Payment_Plan           99995 non-null   object 
 10  Discount_Applied       100000 non-null  int64  
 11  Support_Tickets        100000 non-null  int64  
 12  Feature_Adoption       100000 non-null  int64  
 13  Last_Login             100000 non-null  object 
 14  Churn_Status           100000 non-nul

In [229]:
dataset.describe()

,Customer_ID,Order_ID,Product_ID,Revenue_Per_Customer,Discount_Applied,Support_Tickets,Feature_Adoption,Churn_Status,Customer_Tenure,Customer_Satisfaction,Contract_Renewal,Num_Logins_Last_Month
count,99993.000000,99999.000000,100000.000000,99999.000000,100000.00000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2498.455872,50000.171182,10.472070,2527.351314,14.52636,24.47180,59.448800,0.150050,4.998220,3.005540,0.299670,14.508260
std,1442.375827,28868.030051,5.763226,1425.947481,8.66513,14.45266,23.056294,0.357122,2.584694,1.413177,0.458116,8.654375
min,1.000000,1.000000,1.000000,50.000000,0.00000,0.00000,20.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,1249.000000,25000.500000,5.000000,1292.000000,7.00000,12.00000,39.000000,0.000000,3.000000,2.000000,0.000000,7.000000
50%,2500.000000,49999.000000,10.000000,2525.000000,15.00000,24.00000,59.000000,0.000000,5.000000,3.000000,0.000000,15.000000
75%,3746.000000,75000.500000,15.000000,3760.500000,22.00000,37.00000,79.000000,0.000000,7.000000,4.000000,1.000000,22.000000
max,5000.000000,100000.000000,20.000000,4999.000000,29.00000,49.00000,99.000000,1.000000,9.000000,5.000000,1.000000,29.000000


In [230]:
# Describing the dataset using object view
dataset.describe(include=["object"])

,Subscription_Date,Product_Name,Category,Subscription_Type,Usage_Frequency,Payment_Plan,Last_Login,Marketing_Channel
count,100000,100000,100000,99998,100000,99995,100000,99999
unique,2738,20,5,4,3,4,426,5
top,2025-01-31,SecureAuth,SaaS,Pro,Weekly,Monthly,31/07/2023,Social Media
freq,104,5099,20120,25189,33715,33362,277,20140


In [231]:
# Checking if there is a nmissing values in the dataset
print(dataset.isnull().values.any())

True


The cehcking of the missing value is True, which means there exist missing values in any of the colum

In [232]:
# Summary of all the columns and the number of missing values for each column
print(dataset.isnull().sum())

Customer_ID              7
Order_ID                 1
Subscription_Date        0
Product_ID               0
Product_Name             0
Category                 0
Subscription_Type        2
Usage_Frequency          0
Revenue_Per_Customer     1
Payment_Plan             5
Discount_Applied         0
Support_Tickets          0
Feature_Adoption         0
Last_Login               0
Churn_Status             0
Customer_Tenure          0
Customer_Satisfaction    0
Marketing_Channel        1
Contract_Renewal         0
Num_Logins_Last_Month    0
dtype: int64


### 3.0 Data Cleansing begins at this point


### a. Handling Missing value
There are missing values in some of the data columns including the Customer ID column. Next we need to handle the missing data.
1. Remove the missing row in the Customer ID column and Order ID as there is no way to resolve which customer or Order has the records.
2. Use imputation method of mean, median and mode of each column to fill the other missing value on other columns.

In [233]:
# Drop the records that has the missing value for Customers and Orders.
dataset = dataset.dropna(subset=["Customer_ID", "Order_ID"])

In [234]:
# Handle missing value using mean, median and mode imputation for numerical and categorical data
dataset["Subscription_Type"] = dataset["Subscription_Type"].fillna(dataset["Subscription_Type"].mode()[0])
dataset["Revenue_Per_Customer"] = dataset["Revenue_Per_Customer"].fillna(round(dataset["Revenue_Per_Customer"].mean(), 1))
dataset["Payment_Plan"] = dataset["Payment_Plan"].fillna(dataset["Payment_Plan"].mode()[0])
dataset["Marketing_Channel"] = dataset["Marketing_Channel"].fillna(dataset["Marketing_Channel"].mode()[0])

In [235]:
# Checking if there is a nmissing values in the dataset
print(dataset.isnull().values.any())

False


### b. Handling Inconsistent Data
There is always inconsistencies with different date format. So we will enforce validations to be sure every date colum has same format.
1. Next step is to check out the two date columns and enforce validation


In [236]:
# We will try and format all the data using mixed attribute to dandle each format seperately
dataset['Subscription_Date'] = pd.to_datetime(dataset['Subscription_Date'], format='mixed').dt.strftime('%Y-%m-%d')


dataset['Last_Login'] = pd.to_datetime(dataset['Last_Login'], format='mixed').dt.strftime('%Y-%m-%d')

### c. Data Sorting
Next, we will sort the dataset using Subscription Date. This is make the dataset easily accesible to find information for any record using the customer Subscription date.

In [237]:
# Sort the dataset using customer date of Subscription
dataset.sort_values(by="Subscription_Date", ascending=True, axis=0, inplace=True)

In [238]:
# Confirm the first 20 records
dataset.head(n=20)

,Customer_ID,Order_ID,Subscription_Date,Product_ID,Product_Name,Category,Subscription_Type,Usage_Frequency,Revenue_Per_Customer,Payment_Plan,Discount_Applied,Support_Tickets,Feature_Adoption,Last_Login,Churn_Status,Customer_Tenure,Customer_Satisfaction,Marketing_Channel,Contract_Renewal,Num_Logins_Last_Month
17078,2623.0,86257.0,2022-01-01,17,CodeFusion,Development Tools,Basic,Monthly,4223.0,Monthly,12,6,98,2023-08-15,0,4,2,Referral,0,5
38349,1928.0,92124.0,2022-01-01,3,SyncVault,SaaS,Basic,Weekly,3580.0,Lifetime,12,12,95,2023-03-08,0,8,3,Referral,0,14
67540,3344.0,37567.0,2022-01-01,11,PredictEdge,Artificial Intelligence,Enterprise,Daily,4014.0,Annual,10,31,96,2024-01-02,0,1,5,Referral,1,20
48950,1612.0,44884.0,2022-01-01,7,DataGuardian,Security,Pro,Weekly,2772.0,Monthly,2,34,81,2023-09-22,0,2,3,Direct,1,19
11234,3088.0,56537.0,2022-01-01,19,ScriptEase,Development Tools,Free,Daily,3138.0,Monthly,27,32,55,2023-01-26,0,8,5,Email,0,19
16537,990.0,83634.0,2022-01-01,17,CodeFusion,Development Tools,Enterprise,Daily,817.0,Annual,26,23,55,2024-12-02,0,9,4,Advertisement,1,25
78370,2358.0,91508.0,2022-01-01,12,DeepMindX,Artificial Intelligence,Pro,Daily,1095.0,Lifetime,16,11,91,2023-08-06,0,8,5,Referral,0,29
5938,3891.0,30060.0,2022-01-01,17,CodeFusion,Development Tools,Enterprise,Monthly,4418.0,Monthly,18,27,87,2023-01-01,0,8,1,Referral,0,25
72072,1230.0,60223.0,2022-01-01,10,LinguoSense,Artificial Intelligence,Enterprise,Monthly,557.0,Monthly,27,45,65,2023-11-15,0,9,5,Advertisement,0,24
45624,188.0,28013.0,2022-01-01,5,SentinelShield,Security,Pro,Weekly,1222.0,Monthly,13,1,84,2023-02-04,1,7,1,Advertisement,1,14


In [239]:
# Check the count of all columns
dataset.count()

Customer_ID              99992
Order_ID                 99992
Subscription_Date        99992
Product_ID               99992
Product_Name             99992
Category                 99992
Subscription_Type        99992
Usage_Frequency          99992
Revenue_Per_Customer     99992
Payment_Plan             99992
Discount_Applied         99992
Support_Tickets          99992
Feature_Adoption         99992
Last_Login               99992
Churn_Status             99992
Customer_Tenure          99992
Customer_Satisfaction    99992
Marketing_Channel        99992
Contract_Renewal         99992
Num_Logins_Last_Month    99992
dtype: int64

### <u>4.0 Data Transformation</u>
This step is done to transform all categorical variables to a numerical encoding for ML Model

In [240]:
# Data Transformation 1

# Define categorical columns
categorical_cols = ['Product_Name', 'Category', 'Subscription_Type', 'Usage_Frequency', 'Payment_Plan', 'Marketing_Channel']

# Initialize the encoder
label_encoder = LabelEncoder()

# Apply encoding to each column separately
for col in categorical_cols:
    dataset[col] = label_encoder.fit_transform(dataset[col])


### <u>Data Export of the cleaned dataset</u>
Lastly, We have concluded the dataset preparation as required. We will now export the dataset.
The dataset is fully ready for analysis and ML modeling

In [241]:
dataset.to_csv(path_or_buf="cognitive_cleaned_dataset.csv", index=False)